In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [42]:
# Создаем объект background subtractor
# Алгоритм сегментации фона/переднего плана на основе гауссовой модели. Одной из важных особенностей этого алгоритма является то, 
# что он выбирает соответствующее количество гауссовского распределения для каждого пикселя. 
fgbg = cv2.createBackgroundSubtractorMOG2()

# Открываем файл или захватываем камеру компьютера
cap = cv2.VideoCapture(0)

# Будем заливать движения зеленым
fill_color = (0, 255, 0)

# Счетчик кадров
cl = 0

while True:
    # Считываем кард с видео
    ret, frame = cap.read()
    
	# Проверка на ошибку в считывании
    if not ret:
        break
	
	# Увеличиваем счетчик кадров
    cl = (cl + 1) % 256
    fgmask = fgbg.apply(frame)
    cv2.imshow('frame', fgmask)    
    # Конвертируем в серые оттенки
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Применяем background subtractor к кадру
    #fgmask = fgbg.apply(gray) 
    
    # Применяем пороговое значение к маске переднего плана
	# Для каждого пикселя применяется одно и то же пороговое значение. 
	# Если значение пикселя меньше порогового значения, оно устанавливается равным 0, 
	# в противном случае оно устанавливается на максимальное значение.
    thresh = cv2.threshold(fgmask, 127, 255, cv2.THRESH_BINARY)[1]
    #thresh = cv2.adaptiveThreshold(fgmask,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,5)
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (5,5)) 
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    # Находим контуры
	# в функции cv.findContours() есть три аргумента, первый - исходное изображение,
	# второй - режим поиска контура, третий - метод аппроксимации контура.
    contours, hierarchy = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Заполняем нужным цветом контуры
    for cnt in contours:
        cv2.drawContours(frame, [cnt], 0, fill_color, -1)
        

    cv2.putText(frame, str(cl), (50, 50),cv2.FONT_HERSHEY_SIMPLEX, 1, (200, 10, 20), 2)
    cv2.imshow('frame', frame)    

    # Закрытие окна
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [40]:

cap = cv2.VideoCapture(0)

fgbg = cv2.createBackgroundSubtractorKNN()
#fgbg = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    fgmask = fgbg.apply(gray)
    
    thresh = cv2.threshold(fgmask, 50, 255, cv2.THRESH_BINARY)[1]
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    
    contours, hierarchy = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        cv2.drawContours(frame, [contour], 0, (0,255,0), 2)
    
    mask = np.zeros_like(frame)
    cv2.fillPoly(mask, contours, (0, 255, 0))
    result = cv2.bitwise_and(frame, mask)
    
    cv2.imshow('Motion Detection', result)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Другой вариант: движение отмечается красными точками

In [38]:

cap = cv2.VideoCapture(0)

fgbg = cv2.createBackgroundSubtractorKNN()

while True:
    ret, frame = cap.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    fgmask = fgbg.apply(gray)
    
    thresh = cv2.threshold(fgmask, 50, 255, cv2.THRESH_BINARY)[1]
    
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (5,5)) 
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    
    contours, hierarchy = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
 
    result = frame.copy()
    
    for contour in contours:
        cv2.drawContours(result, [contour], 0, (0,0,255), -1)
    
    # cv2.imshow('Motion Detection', np.hstack((frame, result)))
    cv2.imshow('Motion Detection', result)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [18]:
cap.release()
cv2.destroyAllWindows()